# Tabular Data Interrogation Investigation

In [ ]:
%load_ext dotenv
%dotenv ../.env

In [ ]:
import pandas as pd
import os
from typing import Dict, List, Optional

In [ ]:
# SINGLE FILE VARIABLES - Update the below as required
file_path = "../data/titanic.csv"
DOWNLOAD_URL = "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"
DB_LOC = "titanic.db"
TABLE_NAME = "titanic"

In [ ]:
if not os.path.exists(file_path):
    import subprocess
    # If the file does not exist, it tries to download it from the URL via curl
    try:
        subprocess.run(["curl", "-o", file_path, DOWNLOAD_URL], check=True)
        print(f"Downloaded file {os.path.basename(file_path)} successfully")
    except Exception as e:
        print(f"Error downloading file {e}")



In [ ]:
def ingest_file(file_path: str):
    try:
        if file_path.endswith(".csv"):
            return pd.read_csv(file_path)
        elif file_path.endswith(".xls") or file_path.endswith(".xlsx"):
            return pd.read_excel(file_path) #TODO Modify to handle multiple sheets
        else:
            raise TypeError("Only csvs and excel files are accepted")
    except Exception as e:
        raise e

df = ingest_file(file_path)

In [ ]:
def ingest_files(file_path: str, sheet_names: Optional[List[str]|str]=None)->pd.DataFrame | Dict[str,pd.DataFrame]:
    try:
        if file_path.endswith(".csv"):
            return pd.read_csv(file_path)
        elif file_path.endswith(".xls") or file_path.endswith(".xlsx"):
            if not sheet_names:
                return pd.read_excel(file_path) #TODO Takes just the first sheet
            elif sheet_names=='all':
                return pd.read_excel(file_path, sheet_name=None)
            else:
                sheets = pd.ExcelFile(file_path).sheet_names
                df_map={}
                for sht in sheet_names:
                    if sht in sheets:
                        df_map[sht] = pd.read_excel(file_path, sheet_name=sht)
                return df_map

        else:
            raise TypeError("Only csvs and excel files are accepted")
    except Exception as e:
        raise e

In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine.base import Engine

engine = create_engine(f"sqlite:///{DB_LOC}")

def create_db_tables(dfs: Dict[str, pd.DataFrame] , engine: Engine, replace: bool = False):
    inspector = inspect(engine)
    existing = inspector.get_table_names()
    for table_name in dfs:
        if replace:
            dfs[table_name].to_sql(table_name.strip().replace(" ","_").lower(), engine, index=False, if_exists="replace")
        elif table_name.strip().replace(" ","_").lower() not in existing:
            dfs[table_name].to_sql(table_name.strip().replace(" ","_").lower(), engine, index=False)


In [ ]:
create_db_tables({TABLE_NAME: df}, engine=engine, replace=True)

In [ ]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM titanic WHERE Age > 25 AND Pclass=1;"))

In [ ]:
from redbox.models.settings import Settings
from langchain.chat_models import init_chat_model

env = Settings()
model = llm=  init_chat_model(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        model_provider="bedrock"
    )

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)

agent_executor = create_sql_agent(model, toolkit=toolkit, verbose=False, agent_executor_kwargs={
                "return_intermediate_steps": True
            })

In [ ]:
# agent_executor.run( "How many women survived?")

In [ ]:
# Example query
agent_executor.invoke({"input", "What proportion of survivors were in class 1?"})

In [ ]:
sample_query="How many of the non-survivors were in their thirtees and female?"
# response = agent_executor({"input", sample_query})["output"]
# print(response)

In [ ]:
result = agent_executor({"input", sample_query})
print(result)

In [ ]:
type(result)

### Testing with an `xls` File

In [ ]:
file_path = "../data/test_xls.xlsx"
DB_LOC = "ons_data.db"

In [ ]:
ons_dict=ingest_files(file_path, sheet_names='all')

In [ ]:

for section in ons_dict:
    print(section)
    print(ons_dict[section].columns)
    print(ons_dict[section].head())
    break

In [ ]:
engine = create_engine(f"sqlite:///{DB_LOC}")

In [ ]:
create_db_tables(ons_dict, engine=engine)

In [ ]:
db = SQLDatabase(engine=engine)
toolkit = SQLDatabaseToolkit(db=db, llm=model)

agent_executor = create_sql_agent(model, toolkit=toolkit, verbose=True, agent_executor_kwargs={
                "return_intermediate_steps": True
            })

In [ ]:
sample_query="Which sections have the highest and lowest overall total turnovers respectively?"
response = agent_executor({"input", sample_query})["output"]
print(response)